# Part 3: LangChain Multi-Agent Systems with LangGraph

This notebook demonstrates LangGraph's powerful multi-agent capabilities. We'll explore state management, sequential workflows, and agent coordination that showcase LangChain's advanced orchestration features.

## Environment Setup

Setting up the environment for LangGraph multi-agent workflows with Azure OpenAI.

In [ ]:
# Environment and configuration setup for LangGraph
import os
import warnings
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from typing import TypedDict, Annotated, List, Dict, Any
import operator

# Import LangGraph and LangChain components
from langgraph.graph import StateGraph, END
from langchain_openai import AzureChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import tool

class LangGraphConfig:
    """Configuration management for LangGraph multi-agent workflows"""
    
    def __init__(self):
        # Load environment variables
        env_path = find_dotenv()
        if env_path:
            load_dotenv(env_path)
            print(f"✅ Loaded environment from: {env_path}")
        else:
            warnings.warn("No .env file found. Using system environment variables only.")
        
        self._load_azure_config()
        self._validate_config()
    
    def _load_azure_config(self):
        """Load Azure OpenAI configuration from environment variables"""
        self.azure_api_key = os.getenv('AZURE_OPENAI_API_KEY')
        self.azure_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')
        self.azure_deployment = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME', 'gpt-4o-mini')
        self.azure_api_version = os.getenv('AZURE_OPENAI_API_VERSION', '2024-05-01-preview')
        self.tavily_api_key = os.getenv('TAVILY_API_KEY')
    
    def _validate_config(self):
        """Validate critical configuration"""
        errors = []
        
        if not self.azure_api_key:
            errors.append("AZURE_OPENAI_API_KEY is required")
        
        if not self.azure_endpoint:
            errors.append("AZURE_OPENAI_ENDPOINT is required")
        
        if errors:
            raise ValueError(f"Configuration errors: {', '.join(errors)}")
        
        print("✅ All LangGraph configuration validated successfully")
    
    def create_llm(self, temperature: float = 0.7) -> AzureChatOpenAI:
        """Create an Azure OpenAI LLM instance with the configuration"""
        return AzureChatOpenAI(
            azure_deployment=self.azure_deployment,
            api_version=self.azure_api_version,
            temperature=temperature,
            azure_endpoint=self.azure_endpoint,
            api_key=self.azure_api_key
        )

# Initialize configuration
config = LangGraphConfig()
print(f"🧠 LangGraph Configuration Loaded")

## 3.1 State Management and Basic Graph

LangGraph's core strength is state management across multiple agents. Let's start by understanding the state structure and creating our first graph.

In [ ]:
# Define custom state for our multi-agent workflow
class ResearchState(TypedDict):
    """State structure for research workflow"""
    messages: Annotated[List[Any], operator.add]  # Message history
    topic: str  # Research topic
    research_data: str  # Collected research
    analysis: str  # Analysis results
    final_report: str  # Final output
    current_agent: str  # Track current agent

# Create specialized LLM instances for different agents
researcher_llm = config.create_llm(temperature=0.3)  # Lower temp for factual research
analyst_llm = config.create_llm(temperature=0.5)     # Medium temp for analysis
writer_llm = config.create_llm(temperature=0.7)      # Higher temp for creative writing

# Define tools for our agents
@tool
def research_tool(query: str) -> str:
    """Simulated research tool that returns research data"""
    research_results = {
        "artificial intelligence": "AI is rapidly advancing with developments in LLMs, computer vision, and robotics. Key trends include multimodal AI, edge computing integration, and ethical AI frameworks.",
        "climate change": "Climate change continues to be a critical global challenge. Recent data shows accelerating ice melt, rising sea levels, and increased frequency of extreme weather events.",
        "quantum computing": "Quantum computing is approaching practical applications. IBM, Google, and other companies have achieved quantum advantage in specific tasks, with focus on error correction and scaling."
    }
    
    for topic, data in research_results.items():
        if topic.lower() in query.lower():
            return f"Research findings on {topic}: {data}"
    
    return f"Limited research data available for: {query}"

@tool  
def analysis_tool(data: str) -> str:
    """Simulated analysis tool that processes research data"""
    if "AI" in data or "artificial intelligence" in data.lower():
        return "Analysis: AI sector shows exponential growth with significant investment in enterprise applications and ethical frameworks."
    elif "climate" in data.lower():
        return "Analysis: Climate data indicates urgent need for mitigation strategies and adaptation measures across all sectors."
    elif "quantum" in data.lower():
        return "Analysis: Quantum computing approaching commercialization with potential disruption in cryptography, optimization, and simulation."
    else:
        return f"General analysis: The provided data suggests emerging trends that require further investigation."

print("🤖 Multi-Agent System Initialized")
print("   👥 Agents: Researcher, Analyst, Writer")
print("   🧠 LLMs configured with specialized temperatures")
print("   🛠️ Tools: research_tool, analysis_tool")
print("   📊 State management ready")

## 3.2 Agent Function Definitions

Each agent function operates on the shared state and performs its specialized role.

In [ ]:
def researcher_agent(state: ResearchState) -> ResearchState:
    """Research agent that gathers information on the topic"""
    print(f"🔍 Researcher Agent activated for topic: {state['topic']}")
    
    # Use the research tool to gather information
    research_data = research_tool.invoke(state['topic'])
    
    print(f"   📊 Research completed: {len(research_data)} characters of data")
    
    return {
        **state,
        "research_data": research_data,
        "current_agent": "researcher",
        "messages": state["messages"] + [AIMessage(content=f"Research completed on {state['topic']}.")]
    }

def analyst_agent(state: ResearchState) -> ResearchState:
    """Analyst agent that processes and analyzes research data"""
    print(f"📈 Analyst Agent activated")
    
    if not state.get("research_data"):
        print("   ⚠️ No research data available")
        return {
            **state,
            "analysis": "No research data available for analysis",
            "current_agent": "analyst"
        }
    
    # Use the analysis tool to process research data
    analysis_result = analysis_tool.invoke(state["research_data"])
    
    print(f"   📊 Analysis completed: {len(analysis_result)} characters")
    
    return {
        **state,
        "analysis": analysis_result,
        "current_agent": "analyst",
        "messages": state["messages"] + [AIMessage(content="Analysis phase completed.")]
    }

def writer_agent(state: ResearchState) -> ResearchState:
    """Writer agent that creates the final report"""
    print(f"✍️ Writer Agent activated")
    
    if not state.get("research_data") or not state.get("analysis"):
        print("   ⚠️ Insufficient data for writing")
        return {
            **state,
            "final_report": "Insufficient data to create report",
            "current_agent": "writer"
        }
    
    # Create comprehensive report using LLM
    writing_prompt = ChatPromptTemplate.from_template(
        "You are a skilled technical writer. Create a comprehensive report based on:\n\n"
        "RESEARCH DATA:\n{research_data}\n\n"
        "ANALYSIS:\n{analysis}\n\n"
        "Create a well-structured report with executive summary, key findings, and recommendations."
    )
    
    writing_chain = writing_prompt | writer_llm | StrOutputParser()
    
    final_report = writing_chain.invoke({
        "research_data": state["research_data"],
        "analysis": state["analysis"]
    })
    
    print(f"   📝 Report completed: {len(final_report)} characters")
    
    return {
        **state,
        "final_report": final_report,
        "current_agent": "writer",
        "messages": state["messages"] + [AIMessage(content="Final report completed.")]
    }

print("🤖 Agent Functions Defined:")
print("   🔍 researcher_agent: Gathers information using research tools")
print("   📈 analyst_agent: Processes data using analysis tools")
print("   ✍️ writer_agent: Creates final reports from research and analysis")

## 3.3 Building the LangGraph Workflow

Now let's create the actual LangGraph workflow that connects our agents through a state graph.

In [ ]:
# Create the LangGraph workflow
workflow = StateGraph(ResearchState)

# Add nodes (agents) to the graph
workflow.add_node("researcher", researcher_agent)
workflow.add_node("analyst", analyst_agent)
workflow.add_node("writer", writer_agent)

# Define the workflow edges (sequence)
workflow.set_entry_point("researcher")
workflow.add_edge("researcher", "analyst")
workflow.add_edge("analyst", "writer")
workflow.add_edge("writer", END)

# Compile the graph into a runnable workflow
app = workflow.compile()

print("🔗 LangGraph Workflow Created:")
print("   📊 State: ResearchState with message history and data tracking")
print("   🔀 Flow: Researcher → Analyst → Writer → END")
print("   🤖 Multi-agent coordination with shared state")

# Visualize the workflow structure
print("\n📋 Workflow Structure:")
print("   1. 🔍 Researcher: Gathers information using research tools")
print("   2. 📈 Analyst: Processes and analyzes the research data")
print("   3. ✍️ Writer: Creates comprehensive report from findings")
print("   4. ✅ END: Final state with complete research report")

## 3.4 Executing the Multi-Agent Workflow

Let's run our LangGraph workflow with a research topic and see the agents collaborate.

In [ ]:
async def run_research_workflow(topic: str):
    """Execute the multi-agent research workflow"""
    
    print(f"🚀 Starting Multi-Agent Research Workflow")
    print(f"📝 Topic: {topic}")
    print("=" * 60)
    
    # Initialize the state
    initial_state = {
        "messages": [HumanMessage(content=f"Research topic: {topic}")],
        "topic": topic,
        "research_data": "",
        "analysis": "",
        "final_report": "",
        "current_agent": ""
    }
    
    # Execute the workflow
    try:
        final_state = await app.ainvoke(initial_state)
        
        print("\n" + "=" * 60)
        print("✅ WORKFLOW COMPLETED SUCCESSFULLY")
        print("=" * 60)
        
        print(f"\n📊 Research Data ({len(final_state['research_data'])} chars):")
        print(final_state['research_data'][:200] + "...")
        
        print(f"\n📈 Analysis ({len(final_state['analysis'])} chars):")
        print(final_state['analysis'])
        
        print(f"\n📝 Final Report ({len(final_state['final_report'])} chars):")
        print(final_state['final_report'][:300] + "...")
        
        return final_state
        
    except Exception as e:
        print(f"❌ Workflow error: {e}")
        return None

# Test the workflow with different topics
test_topics = [
    "artificial intelligence",
    "climate change", 
    "quantum computing"
]

for topic in test_topics:
    print(f"\n{'='*60}")
    result = await run_research_workflow(topic)
    if result:
        print(f"✅ Successfully completed research on: {topic}")
    else:
        print(f"❌ Failed to complete research on: {topic}")
    print(f"{'='*60}")

## Summary: LangGraph Multi-Agent Systems

This notebook demonstrated LangGraph's advanced multi-agent capabilities:

### Key Concepts Learned:
1. **State Management**: Shared state across multiple agents with TypedDict definitions
2. **Agent Coordination**: Sequential workflow with specialized agent roles
3. **Graph Construction**: Building workflows with nodes (agents) and edges (transitions)
4. **Tool Integration**: Agents using tools within the multi-agent context

### LangGraph's Advantages:
- **Stateful Workflows**: Persistent state across agent interactions
- **Complex Orchestration**: Support for cycles, conditionals, and dynamic routing
- **Agent Specialization**: Different LLM configurations per agent role
- **Scalable Architecture**: Easy to add new agents and modify workflows

### LangGraph vs Traditional Agents:
| Aspect | Traditional Agents | LangGraph |
|--------|-------------------|----------|
| **State Management** | Limited to single agent | Shared across all agents |
| **Workflow Control** | Linear or simple branching | Complex graphs with cycles |
| **Agent Coordination** | Manual orchestration | Automatic state-driven flow |
| **Debugging** | Limited visibility | Full state inspection at each step |

### Next Steps:
- **Supervisor Patterns**: Central agent coordinating multiple specialists
- **Conditional Routing**: Dynamic agent selection based on state
- **Human-in-the-Loop**: Interactive workflows with user intervention
- **Memory Integration**: Persistent memory across workflow executions

### Production Considerations:
- Implement proper error handling and recovery mechanisms
- Add logging and monitoring for multi-agent workflows
- Consider token usage optimization across multiple LLM calls
- Design state schemas for complex, real-world use cases